In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy 
import json
import csv
import timeit
import time
from datetime import datetime
from bs4 import BeautifulSoup
import re

In [2]:
consumer_key = 'caS0q9PTWurgvlRzlPUbjvKPd'
consumer_secret = 'ZOfPz627CepLQAoNcT4Q3YTH6a2bOussTYCA8ti13P5FMKXJPn'
access_token = '114510363-3XlfxjFi7cZjVjugcJN6tZt3JLnofP1pFIQd6wmW'
access_secret = 'ON79smKdHQkNwxMvDsclgZdHfYX5guJNpqeInw0vosuAQ'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)


In [3]:
tweet = api.get_status(id= 891689557279858688)
tweet.text

u'This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ'

In [4]:
#for status in tweepy.Cursor(api.home_timeline).items():
    # Process a single status
#    print(status.text)

### Gather

In [5]:
prediction_file = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

resp = requests.get(prediction_file)


In [6]:
decod_file = resp.content.decode('utf-8')
file = csv.reader(decod_file.splitlines(), delimiter= '\t')

In [7]:
with open("image-predictions.tsv", 'wb') as f:
    fwriter = csv.writer(f)
    for line in file:
         fwriter.writerow(line)
    

In [8]:
prediction = pd.read_csv("image-predictions.tsv")
twit_archive = pd.read_csv("twitter-archive-enhanced.csv")


#### Calling twitter API for download JSON data for each tweet_id ( API Calling Rate of 800 Calls/15 minutes)

In [11]:
tweet_dataList = []
def correct_tweetID(data, tweet_dataList):
    ctr = 0
    ctr2 = 0
    start = timeit.default_timer()
    print "Start Time: ", start
    for pos,tid in enumerate(data.tweet_id):
        try: 
            tweet_data = api.get_status(id= tid)
            tweet_dataList.append(tweet_data._json)
            ctr +=1
            ctr2 +=1
            if ctr == 800:
                ctr = 0
                print "Sleeping"
                timeshift = timeit.default_timer() - start
                print timeshift
                if timeshift < (15*60):
                    time.sleep( 15*60  - timeshift + 10)
                    print "AWAKE"
                    start = timeit.default_timer()

        ## Id Tweet calls fails, the tweet _id is extracted from twitter text
        except Exception as e:

            print e
            r = re.compile('(\d{18})')      ## Creating regex for 18-digit Tweet_ID
            s = re.search(r, data.loc[pos,'expanded_urls'])     ## Searching and extracting tweet_id from corresponding expanded_url
            correct_id = s.group(0)
            print "Old ID: ", tid
            data.tweet_id = data.tweet_id.replace(tid, correct_id)          ## Replacing incorrect ID with correct ID
            print "New iD: ", data.loc[pos,'tweet_id']
            tweet_data = api.get_status(id= correct_id)          ## API calling with corrected tweet_id to extract JSON data
            tweet_dataList.append(tweet_data._json)
            print "ID corrected And Data downloaded"
            print ctr, ctr2,  " TIME: ", timeit.default_timer()

#correct_tweetID(twit_archive)


#### Saving downloaded JSON data to text file.

In [12]:
with open(" tweet_json.txt", 'w') as f:
    json.dump(tweet_dataList,f)

## Assess

#### Analysing twitter_archieve table

In [ ]:
twit_archive.head()

#### 1. Inspecting for datatype and null value counts

In [ ]:
twit_archive.info()

It can be seen that few columns has large amount of NULL values, thesee correspond to retweets and in-reply-to tweets, which are not actual tweet of WeRateDog twitter handle

#### 2. Inspecting Source values frequency count

In [ ]:
twit_archive.source.value_counts()

#### 3. Inspecting rating Numerator and Denominator

In [ ]:
twit_archive.rating_denominator.value_counts()

In [ ]:
twit_archive.rating_numerator.value_counts()

It has been found that both numerator and denominator has very wiered values, hence it has to matched with those values in the tweet text.

#### Checking accuracy of rating_num and rating_denom

In [ ]:
marksstr = re.compile("(\d{1,3}\.{0,1}\d{0,4})\/(\d{1,3})") ## Generating Regex for extracting rating numerator score from text
for i, text in enumerate(twit_archive.text):
    e_num =  re.search(marksstr, text).group(1)
    num = twit_archive.loc[i,'rating_numerator']
    if int(num) != int(float((e_num))):
        print num, e_num, text

In [ ]:
## Extracting rating denominator score which are incorrect

marksstr = re.compile("(\d{1,3}\.{0,1}\d{0,4})\/(\d{1,3})")
for i, text in enumerate(twit_archive.text):
    e_num =  re.search(marksstr, text).group(2)
    num = twit_archive.loc[i,'rating_denominator']
    if int(num) != int(float((e_num))):
        print num, e_num, text

Hence, it is found that there is some mismatch in the rating-numerator, but not in the rating denominator. Some of values may be looked weired, but that has to be kept as they are extracted from the tweet-text itself. The only examinaotion can be done to re-download the tweet-text dat to check the validity/ accurac of the text.

#### 4. Checking the counts of Dog-Stages i.e. doggo, puppo, pupper and floofer

In [ ]:
print twit_archive.doggo.value_counts(), "\n"
print twit_archive.puppo.value_counts(), '\n'
print twit_archive.floofer.value_counts(),'\n'
print twit_archive.pupper.value_counts()

#### 5. Asserting Dog-stage labels with the corresponding tweet text
Now i will programmitically find by reading each tweet to check correct mapping of the dog-stage is done or  not

##### Inspecting for 'Doggo'


In [ ]:
ctr = 0
find_doggo = twit_archive.text.str.find("doggo")
print "index\tPosition\t tweet"
for i,pos in enumerate(find_doggo):
    if pos != -1:
        print i,"\t", twit_archive.doggo[i],"\t", twit_archive.text[i]
        ctr +=1
print "Total counts: ",ctr

##### Inspecting for 'floofer'
By visual inspection it has been found, floofer dog, is many times refferred as a 'floof' in the tweets, hence will search using 'floof' keyword rather than 'floofer'


In [ ]:
ctr = 0
find_floof = twit_archive.text.str.find("floof")
print "index\tPosition \t tweet"
for i,pos in enumerate(find_floof):
    if pos != -1:
        print i,"\t", twit_archive.floofer[i],"\t", twit_archive.text[i]
        ctr +=1
print "Total counts: ",ctr

##### Inspecting for 'pupper'

In [ ]:
ctr = 0
find_pupper = twit_archive.text.str.find("pupper")
print "index\tPosition \t tweet"
for i,pos in enumerate(find_pupper):
    if pos != -1:
        print i,"\t", twit_archive.pupper[i],"\t", twit_archive.text[i]
        ctr +=1
print "Total counts: ",ctr

##### Inspecting for 'puppo'

In [ ]:
ctr = 0
find_puppo = twit_archive.text.str.find("puppo")
print "index\tDogStage\t tweet"
for i,pos in enumerate(find_puppo):
    if pos != -1:
        print i,"\t", twit_archive.puppo[i],"\t", twit_archive.text[i]
        ctr +=1
print "Total counts: ",ctr

##### Conclusion
Hence it can be concluded, that there many cases for each dog stage which are not correctly mapped and hence is needed to be corrected.

#### 6. Checking Expanded urls

In [ ]:
twit_archive.expanded_urls

#### 7. Tweets which are not from WeRateDogs twitter handle
By visual inspection, it is found that there re many tweets which are from other sources but not WeRateDog twitter handle, hence it is required to programatically access number of such cases.

In [ ]:
twit_archive[twit_archive.expanded_urls.str.find('dog_rates') == -1]

In [ ]:
twit_archive[twit_archive.expanded_urls.isnull()]

There are some non-twitter links and lot of NaNs

#### 8. Assessing tweet JSON data for a sample iD

In [ ]:
tweet = api.get_status(id= 706165920809492480)
tweet._json

In [ ]:
tweet._json.keys()  ### Checking for main keys of the JSON data

#### 9. Checking for accuracy of dog names

In [ ]:
twit_archive[twit_archive.name.str.len() <=2].name

Hence it can be concluded it looks many dogs names are wrongly extracted from the tweet. Especially names like 'a', 'an', 'my' are wrongly chosen

#### 10. Assessing prediction table

In [ ]:
prediction.head()

In [ ]:
prediction.sample(10)

In [ ]:
prediction.info()

In [ ]:
prediction[prediction.p1== 'starfish']

### 1. Quality
#### 1. twitter-archive

  1. The Retweets all correspond to the duplicity of original tweets of WeRateDogs. Hence must be removed.
  2. Tweets corresponding to valid 'in_reply_to_status_id' data are not original review of Dog by WeRateDog, nor it has link for it.
  3. Erroneous data types - Datetime, Dog catagories - Doggo, puppo etc are string object.
  4. Wrongs IDs - Some of the Tweet IDs are wrongly mentioned.
  5. The source column, has a html element of HTTP link. It must have only the text value of html element.
  6. The source column must be of catagory data type
  7. Some of the values of the 'Rating numerator' and 'Rating denominator' has junk values
  8. The dog stage - doggo, floofer, puppo, pupper is also not correctly mapped.
  9. The dog name has sometimes invalid values.
  10. There were lots of NaN value in expanded_url columns.
  11. There is lot of data that is tweets not fron the twitter handle of 'WeRateDogs' but others or from WeRateDog VINE.CO website, which is not a tweet. They all has to be removed as it is required to Wrangle only We Rate Dogs original twitter tweets
  12. tweet_id must be sorted in one direction.

#### 2. prediction table
  1. The tweet_ids must be sorted in one direction : ascending or descending
 
  3. The prediction table must consist of only tweet IDs corresponding to those in twitter_archive_master table.



### 2. Tidyness

  1. The column for retweet IDs and in-reply-to-IDs is not desired and required, as it creates duplicit, and hence be removed
  2. The twitter data for 'retweets_count' and 'likes_count' has to merged with the table.
  3. The dog type columns i.e doggo, floofer, puppo, pupper must be in single column named as Dog type
  4. The prediction column must be part of twitter_archive master table.

In [ ]:
#Creating copy
bc = twit_archive.copy()
twit_archive.info()

## Cleaning

### 1.1.1(Quality) Removing Retweets 
#### Define
Retweets actually creating duplicity in the dataset, as they are retweeting their own tweets. Sometimes they are just retweeting someone else tweets, which is actually not a Dog's Rating. Hence we simply delete all the rows which are the retweets by WeRateDog twitter handle.

#### Code

In [ ]:
# Deleting all the rows which has a valid retweet ID.
bc1 = bc[~bc.retweeted_status_id.notnull()]

#### Test

In [ ]:
bc1.info()

### 1.1.2 (Quality) Removing in_reply_to tweets
#### Define
 Reply tweets also do not are the actual dog ratings tweets, hence they can be removed from the table.

#### Code

In [ ]:
# Deleting rows with valid 'in_reply_to_status_id' data.
bc2 = bc1[~bc1.in_reply_to_status_id.notnull()]

bc2 = bc2.reset_index(drop = True)

#### Test

In [ ]:
bc2.info()

### 2.1 (Tidy) Remove columns correponding to reply_to and re_tweets IDs and status_ids
### 1.1.10 (Quality) Lot of NaN in 'expanded_urls' column in twitter_archive table
#### Define
Since the retweet data and in-reply-to data has been removed, the need for the corresponding column is not required. Hence these column can be easily droped. Such tweets data has all 'NaN' in the expanded_url section. It is assumed that every tweet data must has its url link for reference. Hence the tweets IDs having 'NaN' in expanded must be removed. 


#### Code

In [ ]:
bc2 = bc2.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id',
          'retweeted_status_user_id','retweeted_status_timestamp'], axis = 1)

# Looking for number of tweet IDs with NaN in expanded_url column. 
bc2[bc2.expanded_urls.isnull()]

In [ ]:
bc3 = bc2[~bc2.expanded_urls.isnull()]  #Removing above tweets data from the table
bc3 = bc3.reset_index(drop = True)      #Reseting the index of the data

#### Test

In [ ]:
bc2.info()

In [ ]:
bc3.info()

In [ ]:
bc3.head()

### 1.1.3(Quality) Changing timestamp datatype to Datetime
#### Define
The datetime data column is string-type. But for analysis in later stage programtically, datatime data must be of type 'Dateime' category.

#### Code

In [ ]:
try:
    bc3.timestamp = bc3.timestamp.apply(lambda x: datetime.strptime(x[:-6],'%Y-%m-%d %H:%M:%S'))
except Exception as e:
     print e
             

#### Test

In [ ]:
bc3.info()

### 1.1.4(Quality) Correcting the incorrect tweet IDs
### 2.2(Tidyness)  Extracting 'favorite_count' and 'retweet_count' from the downloaded json data and adding them as column to twitter_archive table.

#### Define
It has been found that some of the tweet IDs are incorrect or are not callable using twitter API. So i will start downloading the json data by calling twitter API one by one. If any failure occur, i will extract the tweet ID from the 'expanded_url' and again make a twitter API call.
The downloaded JSON data will stored in the form of list and is then saved to a text file - 'tweet_json.txt'

#### Code

In [ ]:
with open("tweet_json.txt", 'r') as f:
    json_tweetDataList = json.load(f)

In [ ]:
retweet_count_list = []
favorite_count_list = []
i=0
for jsonData in json_tweetDataList:
    if str(jsonData['id']) == str(bc3.tweet_id[i]) :
        retweet_count_list.append(jsonData["retweet_count"])
        favorite_count_list.append(jsonData["favorite_count"])
        i+=1
       
len(retweet_count_list)


#### Creating column for 'retweet_count' and 'favorite_count'

In [ ]:
bc3["retweet_count"] = pd.Series(retweet_count_list)
bc3["favorite_count"] = pd.Series(favorite_count_list)

In [ ]:
#### Temporarily saving the data to a csv-file
bc3.to_csv("tweet123.csv")


#### Test

In [ ]:
bc3.head(3)

In [ ]:
bc3.info()

### 1.1.5(Quality) The source column, has a html element of HTTP link. It must have only the text value of html element.
### 1.1.6(Quality) The source column must be of type catagory

#### Define
The source column has value represented in XML element. The only important content is the 'text' value of that element. Thid can be done by first converting the string data into html -datatype. And subsequently extracting text value of that element.
Then coverting the datatype of column to 'category' type

#### Code

In [ ]:
##Editing Source

bc3.source = bc3.source.apply(lambda x: BeautifulSoup(x, 'lxml').text)
bc3.source = bc3.source.astype('category')

#### Test

In [ ]:
bc3.source.value_counts()

In [ ]:
bc3.sample(6)

### 1.1.7 (Quality) Correcting Rating: Numerator and Denominator

#### Define
We can extract the correct marks from the tweet status itself and assigned to numerator and denominator column. It has been found that the numerator can be floating point number also. Hence we can create 'Regex' value for different range of values for the numerator and subsequently use it to extract the score from the numerator.

#### Code

In [ ]:
bc3['rating_numerator'] = bc3['rating_numerator'].astype(float)
marksstr = re.compile("(\d{1,3}\.{0,1}\d{0,4})\/(\d{1,3})")
for i, text in enumerate(bc3.text):
    e_num =  re.search(marksstr, text).group(1)
    num = bc3.loc[i,'rating_numerator']
    if int(num) != int(float((e_num))):
        bc3.loc[i, 'rating_numerator'] = e_num
        

In [ ]:
bc3.info()

### 1.1.8 (Quality)Error in correctly mapping of dog catagory 'Floofer' or 'floof', 'doggo' etc in some IDs
It has been found that there are multiple instances, that a dog-stage catagory is present in the tweet-text, slightly in different form, but the corresponding marked as none. This can be done by reading each tweet's ID text, and searching for the desired catagory name in most general form.

#### Code

In [ ]:

for i,txt in  enumerate(bc3.text.str.lower()): ##
    if txt.find("floof") != -1 :
        #print bc3.loc[i, 'floofer'], txt
        bc3.loc[i,'floofer'] = 'floofer'
        bc3.loc[i,'doggo'] = 'None'
        bc3.loc[i,'puppo'] = 'None'
        bc3.loc[i,'pupper'] = 'None'
        
    elif txt.find("doggo") != -1 :
        bc3.loc[i,'doggo'] = 'doggo'
        bc3.loc[i,'puppo'] = 'None'
        bc3.loc[i,'pupper'] = 'None'
        bc3.loc[i,'floofer'] = 'None'
        
    elif txt.find("puppo") != -1 :
        bc3.loc[i,'puppo'] = 'puppo'
        bc3.loc[i,'floofer'] = 'None'
        bc3.loc[i,'doggo'] = 'None'
        bc3.loc[i,'pupper'] = 'None'
        
    elif txt.find("pupper") != -1 :
        bc3.loc[i,'pupper'] = 'pupper'
        bc3.loc[i,'floofer'] = 'None'
        bc3.loc[i,'doggo'] = 'None'
        bc3.loc[i,'puppo'] = 'None'
    
    else:
        bc3.loc[i,'doggo'] = '-'
        bc3.loc[i,'pupper'] = '-'
        bc3.loc[i,'puppo'] = '-'
        bc3.loc[i,'floofer'] = '-'

        

#### Test 
It can be seen from below output, the no. of IDs for which no catagory information be extracted is labelled as '-', and is equal to 1694 for all the cases. The counts of individual dog-stage is also increased from previous one.

In [ ]:
print '\n-- Catagory Doggo -- \n', bc3.doggo.value_counts() 
print '\n-- Catagory puppo -- \n', bc3.puppo.value_counts()
print '\n-- Catagory pupper -- \n', bc3.pupper.value_counts()
print '\n-- Catagory floofer -- \n', bc3.floofer.value_counts()

In [ ]:
bc3.floofer.value_counts()

### 1.1.3(Quality) Dog catagories - Doggo, puppo etc are string object, must be of type catagory

#### Define
Dog-stage type in the table are set as string-type. But for furthur analysis and data exploration, it would be better if it is set as category-type. It can easily done by using 'astype' function in pandas dataframe.

#### Code

In [ ]:
bc3.doggo = bc3.doggo.astype('category')
bc3.pupper = bc3.pupper.astype('category')
bc3.puppo = bc3.puppo.astype('category')
bc3.floofer = bc3.floofer.astype('category')

#### Test

In [ ]:
bc3.info()

##### Saving data in temperory csv file.

In [ ]:
bc3.to_csv('tweetsfilter2.csv')

### 1.1.9 (Quality) Many dogs names are not mapped accurately. Needed to be correct. 

#### Define
For this we have to go through the tweet text data througout the table, and needed to evaluate if there is any consistency in part of text that has dog name present in there. Since there are so many incorrect names extracted, it is required to be done programetically. I am doing this, by re-extracting dog names for all the tweet IDs, whereever possible, though there still must be some errors. 
Based on my analysis of tweet text, i have shortlisted 4 types of string-text, that has name in it.
Such string start as :
1. "named ...."
2. "name is ..."
3. "This is ..." (Excluding "This is a ...")
4. "say hello to ..."

I will look for above strings case by case in each text, and extract the dog name from it.

#### Code

In [ ]:
bc4 = bc3.copy()
nameList = []
for i,text in enumerate(bc4.text):
    #Case 1: Eg - This is a Lofted Aphrodisiac Terrier named Kip. Big fan of bed n breakfasts. Fits perfectly.
    if (text.find('named')) != -1 :
        correct_name = text.split('named ')[1].split('.')[0].split(' ')[0]
    
    #Case 2: Eg- This is my dog. Her name is Zoey. She knows I've been rating other dogs.
    elif (text.find('name is ')) != -1 :
        correct_name = text.split('name is ')[1].split('.')[0].split(' ')[0] 
        #print text
        
    #Case 3: Eg- This is Bruno. He is a service shark. Only gets out of the water to assist you.
    elif (text.split('.')[0].find('This is ')) != -1 :
        #print text
        if (text.find('This is a ')) == -1 :
            correct_name = text.split('This is ')[1].split('.')[0].split(' ')[0]
        else: 
            correct_name = 'None'
    
    #Case 4: Eg- Meet Elliot. He's a Canadian Forrest Pup. Unusual number of antlers for a dog. 
    elif (text.split('.')[0].find('Meet ')) != -1 :
        correct_name = text.split('Meet ')[1].split('.')[0].split(' ')[0]
        #print text
    
    #Case 5: Eg- Say hello to Quinn. She's quite the goofball. Not even a year old.
    elif (text.split('.')[0].lower().find('say hello to ')) != -1 :
        #print text
        correct_name = text.lower().split('say hello to ')[1].split('.')[0].split(' ')[0].title()
        #print correct_name, text
       
    #Case 6: Eg- Here is George. George took a selfie of his new man bun and that is downright epic
    elif (text.split('.')[0].lower().find('here is ')) != -1 :
        #print text
        if len(text.split('.')[0].split(' '))== 3:
            correct_name = text.lower().split('here is ')[1].split('.')[0].split(' ')[0].title()
        else : 
            correct_name = 'None'
            
    
    else: # No name identified
        correct_name = 'None'

    
    nameList.append(re.split('[,.]',correct_name)[0])

bc4['name'] = pd.Series(nameList)
    

#### Test

In [ ]:
ctr = 0
print "Old Name\tCorrected Name"
for i, name in enumerate(bc4.name):
    if bc4.name[i] != bc3.name[i] :
        print bc3.name[i],'\t       ', bc4.name[i], '\t',' ',bc4.text[i]
        ctr +=1
print "Total Correction :", ctr

It can be seen fro mthe output above, most of the wrongly extracted names are corrected with correct name if available or changed to 'None', if no name is available with the total number of correction = 94, which is significant.

### 2.3 (Tidyness) The dog type columns i.e doggo, floofer, puppo, pupper must be merged into single column named as Dog Stage
#### Define
All of the labels of Dog Stage i.e. doggo, floofer, puppo, pupper or None are of same catagory type. Hence thwy must be merged into dsinglw column named 'DogStage' which shows these stages for different IDs. This can be done using pandas 'melt' function

#### Code

In [ ]:
bc5 = pd.melt(bc4, id_vars=['tweet_id','timestamp','source','text','expanded_urls','rating_numerator','rating_denominator','name','retweet_count','favorite_count'], value_vars= ['doggo','floofer','pupper','puppo'], var_name='Dog Stage')

In [ ]:
bc5.info()

##### Since '-' is a case when there is no dog stage available, hence there we multiple (4 times) overlapping of this data. It has to kept only once, and rest all removed

In [ ]:
bc5.drop(bc5[2094:].index[bc5[2094:].value == '-'], inplace= True)  ## Removing extra cases of '-' catagory

In [ ]:
bc5.info()

##### Removing cases, 3 out 4 catagory is 'Null', as it brings huge overlapping

In [ ]:
bc5 = bc5.drop(bc5.index[bc5.value == 'None'])
bc5 = bc5.drop(['Dog Stage'], axis = 1)
bc5 = bc5.rename(columns = {'value':'Dog_Stage'})

#### Test

In [ ]:
bc5.Dog_Stage.value_counts()

In [ ]:
bc5.info()

### 1.1.11 Eliminating all tweets data other than from WeRateDogs
#### Define
Since we have to analyse only tweets (No retweet) of WeRateDog twitter handle, all other tweets must be removed from the table.

#### Code

In [ ]:
bc5[bc5.expanded_urls.str.find('dog_rates') == -1]    
## Extracting tweets which donot have keyword 'dog_rates' in expanded_urls column

In [ ]:
# The above are all those tweets that is not from 'WeRateDog' twitter handle but others.
bc6 = bc5[bc5.expanded_urls.str.find('dog_rates') != -1]
bc6 = bc6.reset_index(drop= True)

##### Test

In [ ]:
bc6.info()

In [ ]:
bc6.to_csv("tweetfilter3.csv")
bc6.tail(5)

### 1.1.12 Sorting twitter_archive table by tweet_id
#### Define
Since all the tweets are jumbled up, they must be sorted in one direction, which help in easy visual/programatic tracing of an ID. So sorting in ascending order.

#### Code

In [ ]:
### 1.2.1 Sorting twitter 
bc6 = bc6.sort_values('tweet_id')

#### Test

In [ ]:
bc6.head(10)

In [ ]:
bc6.tail(10)

In [ ]:
pred = pd.read_csv("image-predictions.tsv")
pred.info()

In [ ]:
pred.head()

### 1.2.1 The prediction table must be sorted w.r.t tweet_id
### 1.2.2 The prediction table must have only tweet IDs corresponding to those in twitter_archive table.
#### Define
The prediction table has 2074 observations, but 'twitter_archive_master.csv' has 1971 observation. Hence the prediction table must have only those IDs which are there in twitter_archive_master table. 
This can be done first extracting those IDs which are their in the master table, and then finally eliminating all other remaining IDs. The final filtered table must be sorted for easy tracebility of data.

#### Code

In [ ]:
## Analysing matching tweet_ids
ctr = 0
id_list = []
for index, tid in enumerate(pred.tweet_id):
    
    if tid in list(bc6.tweet_id):
        id_list.append(index)
        ctr +=1
ctr

In [ ]:
pred2 = pred.iloc[id_list]
pred2 = pred2.reset_index(drop = True)
pred2 = pred2.sort_values('tweet_id')

#### Test

In [ ]:
print pred2.info()
print bc6.info()

pred2.head(10)

### 2.4 One of prediction test must be part of twitter_archive master table
#### Define
The test results of prediction algoritham must be part of twitter_archive_master table for a complete analysis and making data more tidy. Hence i will merged the data of first prediction test i.e. 'p1', 'p1_conf' and 'p1_dog' with the main table.

#### Code

In [ ]:
## Extracting the specific columns for first prediction test
pred2 = pred2.loc[:,['tweet_id', 'p1','p1_conf', 'p1_dog']]

In [ ]:
# Merging the table over the column 'tweet_id'

bc6['tweet_id'] = bc6.tweet_id.astype('int64')  # Ensuring both column have same datatype

twitter_archive_final = pd.merge(bc6, pred2, on= 'tweet_id')


#### Test

In [ ]:
twitter_archive_final.head()

In [ ]:
twitter_archive_final.info()

In [ ]:
### Saving the final master data to csv file

In [ ]:
twitter_archive_final.to_csv('twitter_archive_master.csv')

## Data Insight & Visualization

In [ ]:
tweet_data = pd.read_csv('twitter_archive_master.csv')
pd.options.display.max_colwidth = 100

In [ ]:
tweet_data.info()

In [ ]:
tweet_data.Dog_Stage = tweet_data.Dog_Stage.apply(lambda x: 'ND' if x == '-' else x)
tweet_data.Dog_Stage.value_counts()

#### 1. Tweet with maximum re-tweets

In [ ]:
max_pos = tweet_data.retweet_count.argmax()
 
tweet_data.iloc[max_pos]


#### 2. Tweet with maximum like

In [ ]:
max_pos = tweet_data.favorite_count.argmax()
 
tweet_data.iloc[max_pos]


#### 3. Number of tweets with incorrect Prediction

In [ ]:
tweet_data.p1_dog.value_counts()[1]

Hence no. incorrect predictions = 504

#### 4. Tweet with highest rating and least rating

In [ ]:
rating = tweet_data.rating_numerator/ tweet_data.rating_denominator
print "************************HIGHEST RATING***********************"
tweet_data.iloc[rating.argmax()]

In [ ]:
rating = tweet_data.rating_numerator/ tweet_data.rating_denominator
print "************************LEAST RATING***********************"
tweet_data.iloc[rating.argmin()]

#### 5. Prediction Accuracy of Neural algorithm

In [ ]:
# Predicition accuracy = Total True cases / Total cases
true_cases = tweet_data.p1_dog.value_counts()[1]
total = len(tweet_data)
accuracy = 100.0 *true_cases/total

print "Accuracy = ", accuracy

### Data Visualisation

In [ ]:
from ggplot import *

##### 1. Bar PLot for Dog Stage Category

In [ ]:
data2 = tweet_data[tweet_data.Dog_Stage != 'ND']
ggplot(aes(x ='tweet_data.Dog_Stage'), data = data2) + geom_bar() 

##### 2. Trends for no. of likes vs Time

In [ ]:
#tweet_data
#tweet_data.timestamp = tweet_data.timestamp.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

ggplot(aes(x ='tweet_data.timestamp', y = 'tweet_data.favorite_count'), data = tweet_data) + \
geom_point(alpha =1) + stat_smooth(color = 'blue') +\
scale_x_date(labels='%m-%y')+ \
ylim(0,50000) +\
xlab('Time') + ylab('No. of Likes')

##### 3. Trends for no. of retweets vs Time

In [ ]:
ggplot(aes(x ='tweet_data.timestamp', y = 'tweet_data.retweet_count'), data = tweet_data) +\
geom_point() + stat_smooth(color = 'red') +\
ylim(0,12000) + scale_x_date(labels='%m-%y') +\
xlab('Time') + ylab('No. of Retweets')

It can be concluded from 2,3 and 4 that overall popularity of WeRateDogs twitter page, has increased significantly over time which is indicated by the increase in the amount of average no. of retweets and average no. of like to a given post/tweet.

##### 4. No. of retweets over time for different dog stages 

In [ ]:
ggplot(aes(x ='tweet_data.timestamp', y = 'tweet_data.retweet_count', color = 'Dog_Stage'), data = data2) +\
geom_point() + stat_smooth(se = 'False') +\
scale_color_brewer(type = 'qual', palette = 2) +\
ylim(0,12000) + scale_x_date(labels='%m-%y') +\
xlab('Time') + ylab('No. of Retweets')

##### 5. Rating histogram for different Dog-stages

In [ ]:

ggplot(aes(x ='tweet_data.rating_numerator/ tweet_data.rating_denominator', fill = 'Dog_Stage' ),\
       data = data2) +\
geom_histogram(binwidth = 0.03)  + xlim(0,1.5)+\
xlab('Equivalent Rating') + ylab('Count')

It can be seen that the 'doggo' category has recieved more of the high rating value compared to others. 